In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from tqdm import tqdm

lst = pd.read_csv('/content/drive/MyDrive/deeplearningproject/statute_lst.csv',index_col=0)
main=pd.read_csv('/content/drive/MyDrive/deeplearningproject/statute_main.csv',index_col=0)

In [14]:
lst.head(10)

,법령일련번호,법령명한글,법령명영문,소관부처명,법령구분명
0,253949,119구조ㆍ구급에 관한 법률,ACT ON 119 RESCUE AND EMERGENCY MEDICAL SERVICES,"소방청,소방청,소방청,소방청,소방청",법률
1,228807,119구조ㆍ구급에 관한 법률 시행령,ENFORCEMENT DECREE OF THE ACT ON 119 RESCUE AND EMERGENCY MEDICAL SERVICES,"소방청,소방청,소방청,소방청,소방청",대통령령
2,243115,1인 창조기업 육성에 관한 법률,ACT ON THE FOSTERING OF SELF-EMPLOYED CREATIVE ENTERPRISES,중소벤처기업부,법률
3,245891,1인 창조기업 육성에 관한 법률 시행령,ENFORCEMENT DECREE OF THE ACT ON THE FOSTERING OF SELF-EMPLOYED CREATIVE ENTERPRISES,중소벤처기업부,대통령령
4,191042,"2011대구세계육상선수권대회, 2013충주세계조정선수권대회, 2014인천아시아경기대회, 2014인천장애인아시아경기대회 및 2015광주하계유니버시아드대회 지원법","ACT ON ASSISTANCE TO THE IAAF WORLD CHAMPIONSHIPS DAEGU 2011, THE 2013 WORLD ROWING CHAMPIONSHIPS CHUNGJU, THE 17TH 2014 INCHEON ASIAN GAMES, THE 2014 INCHEON ASIAN PARA GAMES, AND THE 2015 GWANGJU SUMMER UNIVERSIADE",문화체육관광부,법률
5,206430,2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법,SPECIAL ACT ON SUPPORT FOR THE 2018 PYEONGCHANG OLYMPIC AND PARALYMPIC WINTER GAMES,문화체육관광부,법률
6,209598,2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법 시행령,ENFORCEMENT DECREE OF THE SPECIAL ACT ON SUPPORT FOR THE 2018 PYEONGCHANG OLYMPIC AND PARALYMPIC WINTER GAMES,문화체육관광부,대통령령
7,205822,2023 새만금 세계스카우트잼버리 지원 특별법,SPECIAL ACT ON SUPPORT FOR THE 2023 SAEMANGEUM WORLD SCOUT JAMBOREE,여성가족부,법률
8,212307,2023 새만금 세계스카우트잼버리 지원 특별법 시행령,ENFORCEMENT DECREE OF THE SPECIAL ACT ON SUPPORT FOR THE 2023 SAEMANGEUM WORLD SCOUT JAMBOREE,여성가족부,대통령령
9,162682,4ㆍ16세월호참사 진상규명 및 안전사회 건설 등을 위한 특별법,SPECIAL ACT ON INVESTIGATING THE TRUTH OF THE16 SEWOL FERRY DISASTER AND BUILDING A SAFE SOCIETY,해양수산부,법률


In [3]:
main.head()

,법령일련번호,Title,Contents
0,253949,Purpose,Article 1 (Purpose) The purpose of this Act is...
1,253949,Definitions,Article 2 (Definitions) The terms used in this...
2,253949,Responsibilities of the State,Article 3 (Responsibilities of the State) (1) ...
3,253949,Rights and Obligations of People,Article 4 (Rights and Obligations of People) (...
4,253949,Relationship to Other Statutes,Article 5 (Relationship to Other Statutes) Exc...


In [4]:
df=main[['Contents']]
df.head()

,Contents
0,Article 1 (Purpose) The purpose of this Act is...
1,Article 2 (Definitions) The terms used in this...
2,Article 3 (Responsibilities of the State) (1) ...
3,Article 4 (Rights and Obligations of People) (...
4,Article 5 (Relationship to Other Statutes) Exc...


In [5]:
filtered_df = df[~df['Contents'].str.contains('\(Purpose\)|\(Definitions\)')]
filtered_df=filtered_df.sample(n=5000, random_state=1)

In [6]:
import multiprocessing as mp
import numpy as np
import nltk
nltk.download('stopwords',quiet=True)
# 'punkt' 패키지 다운로드
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
pd.set_option('display.max_colwidth', None)

def preprocess_text(text):
    # lowercasing
    lowercased_text=text.lower()

    # cleaning
    import re
    remove_punctuation=re.sub(r'\n','',lowercased_text) # \n
    remove_punctuation=re.sub(r'|\barticle \d+','',remove_punctuation) # 절
    remove_punctuation=re.sub(r'|<amended on .*?>','',remove_punctuation) # 개정년도 문장 삭제
    remove_punctuation=re.sub(r'\[this article newly inserted on .*?\]','',remove_punctuation) # 조문 삽입 문장 삭제 [this article newly inserted on dec. 22, 2015]
    remove_punctuation=re.sub(r'\[this article wholly amended on .*?\]','',remove_punctuation)# [this article wholly amended on mar. 28, 2008]
    remove_punctuation=re.sub(r'|^.*deleted \w+ \d{4}.*$','',remove_punctuation) # 조문 삭제 문장 삭제
    remove_punctuation=re.sub(r'|[^\uAC00-\uD7A30-9a-zA-Z\s]','',remove_punctuation) # 특수문자
    remove_space= remove_punctuation.strip()

    #tokenization
    from nltk.tokenize import word_tokenize
    tokenized_sentence= word_tokenize(remove_space)

    # stop words
    from nltk.corpus import stopwords
    stopwords=set(stopwords.words('english'))

    stopwords_remove= [word for word in tokenized_sentence if word not in stopwords]


    # stemming -> 어간 추출. 표제어 추출은 일일이 설정해야해서 pass
    from nltk.stem import PorterStemmer
    ps=PorterStemmer()
    stemmed_text=[ps.stem(word) for word in stopwords_remove]


    # result
    df=pd.DataFrame({
        'Contents':[text],
        'Lowercase':[lowercased_text],
        'Cleaning':[remove_space],
        'Tokenization':[tokenized_sentence],
        'Stopwords':[stopwords_remove],
        'Stemming':[stemmed_text]
    })

    return df

def preprocessing(corpus):
    df=pd.DataFrame(columns=['Contents'])

    for doc in corpus['Contents']:
      result_df=preprocess_text(doc)

      df=pd.concat([df,result_df],ignore_index=True)

    return df

result=preprocessing(filtered_df)
result

Contents  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Article 68 (Formulation of Phased-Implementation Plan) (1) The Special Metropolitan City Mayor, the Metropolitan City Mayor, the head of Si or the 

In [8]:
def calculate_tfidf(corpus):
    df = preprocessing(corpus)
    # 다시 문장으로 결합
    stemming = corpus['Stemming'].apply(' '.join)

    # TF-IDF 계산
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(stemming)

    return pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

result_tfidf = calculate_tfidf(result)
result_tfidf

,01,025,03,05,10,100,1000,10000,100000,100100,...,zeolit,zero,zeroemiss,zeroenergi,zinc,zirconium,zonal,zone,zoo,zootechn
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4998,0.0,0.0,0.0,0.0,0.014471,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.056706,0.0,0.0


In [9]:
def cosineSimilarity(corpus):
    from sklearn.metrics.pairwise import cosine_similarity
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(result_tfidf, result_tfidf)
    cosine_sim_df = pd.DataFrame(cosine_sim, columns=[f'Doc_{i}' for i in range(len(corpus))], index=[f'Doc_{i}' for i in range(len(corpus))])

    return cosine_sim_df

result_cosim = cosineSimilarity(result_tfidf)
result_cosim

,Doc_0,Doc_1,Doc_2,Doc_3,Doc_4,Doc_5,Doc_6,Doc_7,Doc_8,Doc_9,...,Doc_4990,Doc_4991,Doc_4992,Doc_4993,Doc_4994,Doc_4995,Doc_4996,Doc_4997,Doc_4998,Doc_4999
Doc_0,1.000000,0.028866,0.0,0.017684,0.058012,0.021924,0.011312,0.066163,0.000000,0.000000,...,0.015006,0.049026,0.042576,0.012047,0.013377,0.014011,0.007580,0.028347,0.029215,0.006860
Doc_1,0.028866,1.000000,0.0,0.002905,0.025471,0.083943,0.001753,0.002915,0.000000,0.000000,...,0.042321,0.078576,0.001110,0.035927,0.019473,0.001546,0.001116,0.025836,0.025354,0.019161
Doc_2,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.173597,0.195829,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Doc_3,0.017684,0.002905,0.0,1.000000,0.012357,0.004755,0.015165,0.031627,0.020623,0.023264,...,0.020527,0.024338,0.009809,0.085977,0.065472,0.019176,0.008131,0.138099,0.017172,0.011052
Doc_4,0.058012,0.025471,0.0,0.012357,1.000000,0.023289,0.018449,0.052731,0.000000,0.000000,...,0.025488,0.041219,0.005144,0.041773,0.014591,0.004287,0.015946,0.016672,0.035807,0.014517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc_4995,0.014011,0.001546,0.0,0.019176,0.004287,0.041965,0.001660,0.002761,0.000000,0.000000,...,0.003151,0.030373,0.025611,0.019143,0.008875,1.000000,0.003378,0.055717,0.009879,0.026835
Doc_4996,0.007580,0.001116,0.0,0.008131,0.015946,0.001240,0.003865,0.012895,0.000000,0.000000,...,0.002275,0.006353,0.002426,0.004499,0.004588,0.003378,1.000000,0.004719,0.046823,0.011638
Doc_4997,0.028347,0.025836,0.0,0.138099,0.016672,0.021357,0.009297,0.024288,0.000000,0.000000,...,0.041058,0.052287,0.013400,0.076354,0.065042,0.055717,0.004719,1.000000,0.052401,0.014302
Doc_4998,0.029215,0.025354,0.0,0.017172,0.035807,0.037773,0.022765,0.039111,0.000000,0.000000,...,0.012886,0.083723,0.012051,0.107450,0.219712,0.009879,0.046823,0.052401,1.000000,0.016648


In [11]:
def top3_sim_sentences(result, result_cosim, top_n=3):
    similar_sentences = []
    for idx, row in result.iterrows():
        # 자기 자신을 제외하고 유사도가 높은 인덱스 상위 top_3 개 추출
        similar_index = result_cosim.iloc[idx].drop(f'Doc_{idx}').nlargest(top_n).index
        similar_index = [int(i.split('_')[1]) for i in similar_index]
        similar_sentences.append({
            'Contents': row['Contents'],
            'top 1': result.iloc[similar_index[0]]['Contents'],
            'top 2': result.iloc[similar_index[1]]['Contents'],
            'top 3': result.iloc[similar_index[2]]['Contents'],
            'cosim 1': result_cosim.iloc[idx, similar_index[0]],
            'cosim 2': result_cosim.iloc[idx, similar_index[1]],
            'cosim 3': result_cosim.iloc[idx, similar_index[2]],
        })
    return pd.DataFrame(similar_sentences)

# 각 문장에 대해 유사도가 높은 상위 3개의 문장 찾기
simlaw_df = top3_sim_sentences(result, result_cosim, top_n=3)

simlaw_df.head()

Contents  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Article 68 (Formulation of Phased-Implementation Plan) (1) The Special Metropolitan City Mayor, the Metropolitan City Mayor, the head of Si or the head of Gun shall, when he intends to draw up a phased-implementation plan under Article 58 (1) of the Act, consult in advance with the heads of administrative agencies concerned.\n  (2) The Special Metropolitan City Mayor, the Metropolitan City Mayor, the head of Si or the head of Gun may examine the second-stage implementation plan every year and select urban planning facilities to be installed in an urban planning project planned to be implemented within 3 years to include such urban planning facilities in the first-stage implementation plan.\n  (3) The publication of the phased implementation plan under Article 58 (4) of the Act shall be made in the bulletin issued by the local government concerned.\n  (4) The term “minor matter prescribed by the Presidential Decree” in the proviso of Article 58 (5) of the Act means a change in the phased implementation plan following a minor change in the urban planning under each subparagraph of Article 24 (3) and (4).   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [12]:
# csv 파일 저장
simlaw_df.to_csv('/content/drive/MyDrive/deeplearningproject/simlaw_df.csv', index=False)

문장 유사도 결과 값을 보고 고민해보아야 할 점
- Article 17-3 (Emergency Measures) 조문만 나올 것이 아니라 무슨법의 무슨 조문으로 표시가 되어야 할 듯 하다.
- 영어 해석의 문제^^;
- 119 구조 관련 키워드가 던져지면 그 법과 시행령이 나와야 정상일듯?